### Data processing

In this file, I prepare data from the [Kaggle dataset](https://www.kaggle.com/datasets/omermetinn/tweets-about-the-top-companies-from-2015-to-2020/data) for use on the Dashboard. The original dataset is too big(4M tweets and more than 700MB) for learning purposes, so I am going to use only tweets from 2019. The original dataset is not committed to GitHub; you can additionally load it from Kaggle.

In [31]:
import pandas as pd
from pathlib import Path
import os

tweet_file = Path("../data/Tweet.csv")
company_tweet_file = Path("../data/Company_Tweet.csv")
company_file = Path("../data/Company.csv")
output_file  = Path("../data/prepared_data.csv")

if os.path.isfile(tweet_file) and os.path.isfile(company_tweet_file):
    # Load the CSV file
    tweets = pd.read_csv(tweet_file)
    company_tweet = pd.read_csv(company_tweet_file)
    # Merge files
    tweets = tweets.merge(company_tweet, how='left', on='tweet_id')
    # Format dates
    tweets['date'] = pd.to_datetime(tweets['post_date'], unit='s').dt.date
    tweets.date = pd.to_datetime( tweets.date,errors='coerce')
    tweets['time'] = pd.to_datetime(tweets['post_date'], unit='s').dt.time

I am going to use only 2019 year's tweets, because this dataset too big.

In [32]:
tweets_2019 = tweets[tweets['date'].dt.year == 2019]
print(len(tweets))
print(len(tweets_2019))

4336445
905311


We can see than selected dataset in 4 times smaller than the original one.

Add company name's to dataset:

In [33]:
if os.path.isfile(company_file):
    company = pd.read_csv(company_file)
    tweets_2019 = tweets_2019.merge(company, how='left', on='ticker_symbol')

Drop columns, which value doesn't necessary for analysis:

In [34]:
tweets_2019.head()

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol,date,time,company_name
0,1079890068867817473,evdefender,1546300830,!! 8 Hours Left !!The picture you see here is...,6,16,82,TSLA,2019-01-01,00:00:30,Tesla Inc
1,1079890105282842629,ExactOptionPick,1546300839,Don't miss our next FREE OPTION TRADE. Sign u...,0,0,0,GOOGL,2019-01-01,00:00:39,Google Inc
2,1079890627335213057,traderDanielle,1546300963,Rinse and repeat - looking for entries in $XLC...,0,0,3,GOOGL,2019-01-01,00:02:43,Google Inc
3,1079890718326431745,O__rust,1546300985,$3750 would be the minimum discount in any cas...,0,0,0,TSLA,2019-01-01,00:03:05,Tesla Inc
4,1079890782742503424,GunGermSteel,1546301000,There are FUDking analysts like those from gol...,0,0,1,TSLA,2019-01-01,00:03:20,Tesla Inc


In [35]:
tweets_2019 = tweets_2019.drop('tweet_id', axis=1)
tweets_2019 = tweets_2019.drop('writer', axis=1)
tweets_2019 = tweets_2019.drop('post_date', axis=1)
tweets_2019 = tweets_2019.drop('ticker_symbol', axis=1)

In [36]:
list(tweets_2019)

['body',
 'comment_num',
 'retweet_num',
 'like_num',
 'date',
 'time',
 'company_name']

In [37]:
tweets_2019.describe()

,comment_num,retweet_num,like_num,date
count,905311.000000,905311.000000,905311.000000,905311
mean,0.604147,0.901069,4.928085,2019-06-20 00:12:43.112786432
min,0.000000,0.000000,0.000000,2019-01-01 00:00:00
25%,0.000000,0.000000,0.000000,2019-03-21 00:00:00
50%,0.000000,0.000000,1.000000,2019-06-05 00:00:00
75%,0.000000,0.000000,2.000000,2019-09-20 00:00:00
max,567.000000,989.000000,995.000000,2019-12-31 00:00:00
std,3.207685,6.756438,24.012596,NaN


### Identifying Missing Values

In [38]:
round((tweets_2019.isnull().sum()*100 / tweets_2019.shape[0]),2)

body            0.0
comment_num     0.0
retweet_num     0.0
like_num        0.0
date            0.0
time            0.0
company_name    0.0
dtype: float64

There are no missed values. 

### Prepare tweets body

In [39]:
import re

replacement_text="[URL REMOVED]"
url_pattern = re.compile(r'https?://\S+|www\.\S+')

tweets_2019['body'] = tweets_2019['body'].str.replace('$', '')
tweets_2019['body'] = tweets_2019['body'].apply(lambda x: url_pattern.sub(replacement_text, x))

tweets_2019.head()

,body,comment_num,retweet_num,like_num,date,time,company_name
0,!! 8 Hours Left !!The picture you see here is...,6,16,82,2019-01-01,00:00:30,Tesla Inc
1,Don't miss our next FREE OPTION TRADE. Sign u...,0,0,0,2019-01-01,00:00:39,Google Inc
2,"Rinse and repeat - looking for entries in XLC,...",0,0,3,2019-01-01,00:02:43,Google Inc
3,3750 would be the minimum discount in any case...,0,0,0,2019-01-01,00:03:05,Tesla Inc
4,There are FUDking analysts like those from gol...,0,0,1,2019-01-01,00:03:20,Tesla Inc


### Add Engagement mentric for tweet

Combine fields 'comment_num', 'retweet_num' and 'like_num' to one field with diffirent wieghts:

In [45]:
tweets_2019 = pd.read_csv(output_file)
tweets_2019['engagement'] = tweets_2019['comment_num'] * 3 + tweets_2019['retweet_num'] * 2 + tweets_2019['like_num']
tweets_2019 = tweets_2019.drop('comment_num', axis=1)
tweets_2019 = tweets_2019.drop('retweet_num', axis=1)
tweets_2019 = tweets_2019.drop('like_num', axis=1)

tweets_2019.to_csv(output_file, index=False)

### Sentiment Analysis

In [40]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('vader_lexicon')

from nltk.sentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/work/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [41]:
import numpy as np
sia = SentimentIntensityAnalyzer()

def get_sentiment(tweets):
    # applt the SentimentIntensityAnalyzer
    tweets.loc[:,('score')]=tweets.loc[:,'body'].apply(lambda x: sia.polarity_scores(x)['compound'])
    # create label
    #bins= pd.interval_range(start=-1, freq=3, end=1)
    tweets.loc[:,('sentiment')]=pd.cut(np.array(tweets.loc[:,'score']),bins=[-1, -0.66, 0.32, 1],right=True ,labels=('negative', 'neutral', 'positive'))
    
    df=tweets.loc[:,["date","score","sentiment","body"]]
    return df

print('apple misses earnings, analyst suggest downgrade , sell now ')
sia.polarity_scores('apple misses earnings, analyst suggest downgrade , sell now ')

apple misses earnings, analyst suggest downgrade , sell now 


{'neg': 0.213, 'neu': 0.787, 'pos': 0.0, 'compound': -0.2263}

In [42]:
# augment vocab

positive_words='high profit Growth Potential Opportunity Bullish Strong Valuable Success Promising Profitable Win Winner Outstanding Record Earnings Breakthrough buy bull long support undervalued underpriced cheap upward rising trend moon rocket hold breakout call beat support buying holding free ready'
negative_words='resistance squeeze cover seller Risk Loss Decline Bearish Weak Declining Uncertain Troubling Downturn Struggle Unstable Volatile Slump Disaster Plunge sell bear bubble bearish short overvalued overbought overpriced expensive downward falling sold sell low put miss'

dictOfpos = { i : 4 for i in positive_words.split(" ") }
dictOfneg = { i : -4 for i in negative_words.split(" ")  }
Financial_Lexicon = {**dictOfpos, **dictOfneg}

sia.lexicon.update(Financial_Lexicon)

print('apple misses earnings, analyst suggest downgrade , sell now ')
print(sia.polarity_scores('apple misses earnings, analyst suggest downgrade , sell now '))

tw=get_sentiment(tweets_2019) # get tweets
tw.head()

apple misses earnings, analyst suggest downgrade , sell now 
{'neg': 0.535, 'neu': 0.465, 'pos': 0.0, 'compound': -0.7845}


,date,score,sentiment,body
0,2019-01-01,-0.5216,neutral,!! 8 Hours Left !!The picture you see here is...
1,2019-01-01,0.9493,positive,Don't miss our next FREE OPTION TRADE. Sign u...
2,2019-01-01,0.6428,positive,"Rinse and repeat - looking for entries in XLC,..."
3,2019-01-01,0.0875,neutral,3750 would be the minimum discount in any case...
4,2019-01-01,0.3612,positive,There are FUDking analysts like those from gol...


### Save data for dashboard

In [43]:
tweets_2019.to_csv(output_file, index=False)

In [100]:
tweets_2019 = pd.read_csv(output_file)
tweets_2019.iloc[:len(tweets_2019)//2,:].to_csv(f'{output_file}_1', index=False)
tweets_2019.iloc[len(tweets_2019)//2:,:].to_csv(f'{output_file}_2', index=False)